In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

/home/maria/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# load in raw counts
counts_df_raw = pd.read_csv("../data/bottomly/bottomly_count_table.txt",sep="\t").set_index('gene')
counts_df = counts_df_raw[(counts_df_raw.values>0).all(axis=1)]  # remove all genes that had non-zero reads for any sample... low quality...
counts_df = counts_df.dropna() 
metadata = pd.read_csv("../data/bottomly/bottomly_phenodata.txt",sep=" ")

In [8]:
# add geoMean
counts_ = counts_df.values

counts_df.loc[len(counts_df)] = np.exp( (1/counts_.shape[0])*np.log(counts_).sum(axis=0) )
counts_df = counts_df.rename(index={counts_df.index[-1]: "GeoMean"})
counts_df

,SRX033480,SRX033488,SRX033481,SRX033489,SRX033482,SRX033490,SRX033483,SRX033476,SRX033478,SRX033479,...,SRX033473,SRX033474,SRX033475,SRX033491,SRX033484,SRX033492,SRX033485,SRX033493,SRX033486,SRX033494
gene,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000000001,369.000000,744.000000,287.000000,769.000000,348.000000,803.000000,433.00000,469.000000,585.000000,321.000000,...,461.000000,309.000000,374.000000,781.000000,555.000000,820.000000,294.000000,758.000000,419.000000,857.000000
ENSMUSG00000000056,21.000000,46.000000,20.000000,36.000000,12.000000,55.000000,27.00000,44.000000,32.000000,47.000000,...,40.000000,30.000000,27.000000,46.000000,28.000000,40.000000,21.000000,52.000000,27.000000,45.000000
ENSMUSG00000000058,15.000000,43.000000,12.000000,34.000000,14.000000,32.000000,19.00000,18.000000,44.000000,22.000000,...,24.000000,29.000000,15.000000,34.000000,23.000000,38.000000,17.000000,29.000000,12.000000,28.000000
ENSMUSG00000000078,517.000000,874.000000,340.000000,813.000000,378.000000,860.000000,528.00000,401.000000,584.000000,401.000000,...,431.000000,341.000000,480.000000,930.000000,585.000000,1137.000000,490.000000,1079.000000,565.000000,726.000000
ENSMUSG00000000088,273.000000,781.000000,275.000000,745.000000,301.000000,653.000000,365.00000,414.000000,454.000000,331.000000,...,548.000000,413.000000,395.000000,1153.000000,878.000000,859.000000,431.000000,909.000000,592.000000,1009.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000090236,2.000000,18.000000,2.000000,11.000000,2.000000,8.000000,10.00000,4.000000,3.000000,1.000000,...,9.000000,3.000000,6.000000,12.000000,9.000000,17.000000,5.000000,7.000000,2.000000,5.000000
ENSMUSG00000090262,53.000000,82.000000,46.000000,84.000000,45.000000,109.000000,55.00000,73.000000,63.000000,94.000000,...,80.000000,90.000000,55.000000,120.000000,54.000000,98.000000,54.000000,112.000000,54.000000,85.000000
ENSMUSG00000090264,1.000000,4.000000,3.000000,6.000000,1.000000,7.000000,2.00000,18.000000,9.000000,5.000000,...,15.000000,5.000000,3.000000,11.000000,2.000000,6.000000,3.000000,7.000000,2.000000,6.000000


In [9]:
np.savetxt(f'../data/bottomly/nonzero_gene_counts.txt',counts_df.values, delimiter='\t', fmt='%d')

In [15]:
# various downsampling approaches -- fit on 10, 25, 50, 75 % of genes, randomly selected
percentages = [10,25,50,75]
for p in percentages:
    num = int(p*.01*9196)
    filt = np.random.choice(np.arange(9196),size=num)
    counts_df_ = counts_df.iloc[filt,:]
    counts_ = counts_df_.values

    counts_df_.loc[len(counts_df_)] = np.exp( (1/counts_.shape[0])*np.log(counts_).sum(axis=0) )
    counts_df_ = counts_df_.rename(index={counts_df_.index[-1]: "GeoMean"})
    np.savetxt(f'../data/bottomly/nonzero_gene_counts_{p}p.txt',counts_df.values, delimiter='\t', fmt='%d')

/tmp/ipykernel_2624551/1185652138.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counts_df_.loc[len(counts_df_)] = np.exp( (1/counts_.shape[0])*np.log(counts_).sum(axis=0) )
/tmp/ipykernel_2624551/1185652138.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counts_df_.loc[len(counts_df_)] = np.exp( (1/counts_.shape[0])*np.log(counts_).sum(axis=0) )
/tmp/ipykernel_2624551/1185652138.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

In [20]:
# various downsampling approaches -- halve 10, 25, 50, 75, 90 % of genes, randomly selected
percentages = [10,25,50,75,90]
for p in percentages:
    num = int(p*.01*9196)
    filt = np.random.choice(np.arange(9196),size=num)
    counts_df_ = counts_df.copy()
    counts_df_.iloc[filt,:] = counts_df.iloc[filt,:]*.5
    counts_ = counts_df_.values
    counts_df_.loc[len(counts_df_)] = np.exp( (1/counts_.shape[0])*np.log(counts_).sum(axis=0) )
    counts_df_ = counts_df_.rename(index={counts_df_.index[-1]: "GeoMean"})
    np.savetxt(f'../data/bottomly/nonzero_gene_counts_{p}p_.5f.txt',counts_df.values, delimiter='\t', fmt='%d')

In [ ]:
# run deseq2, edgeR, and glmFit workflow on all samples